<a href="https://colab.research.google.com/github/fahimku2020/fahimku2020/blob/main/Good_semantic_similarity_search_wikipedia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install wikipedia
!pip install sentence-transformers

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=2f13e2df93714c7923aa02326c8483c85092fd6f898773dd4503a4a54fb83e3d
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [5]:
import nltk
nltk.download('punkt') #Ensure Punkt Sentence Tokenizer is downloaded
nltk.download('punkt_tab')

from sentence_transformers import SentenceTransformer, util
import wikipedia
import re

def get_sentences_from_wikipedia(topic, num_sentences=100):
    """Fetches sentences from Wikipedia and returns a list."""
    try:
        page = wikipedia.page(topic)
        text = page.content
        sentences = nltk.sent_tokenize(text)
        #Remove sentences with excessive numbers or special characters (Noise reduction)
        cleaned_sentences = [s for s in sentences if len(re.findall(r'\d+', s)) < 5 and len(re.findall(r'[^a-zA-Z0-9\s]', s)) < 10 ]
        return cleaned_sentences[:min(num_sentences, len(cleaned_sentences))]
    except wikipedia.exceptions.PageError:
        print(f"Wikipedia page not found for '{topic}'.")
        return []
    except wikipedia.exceptions.DisambiguationError as e:
        print(f"Disambiguation error for '{topic}': {e.options}")
        return []


def semantic_similarity_query(topic, query_sentence):
    """Builds semantic similarity and performs a query."""
    sentences = get_sentences_from_wikipedia(topic)
    if not sentences:
        return []

    model = SentenceTransformer('all-mpnet-base-v2')
    embeddings = model.encode(sentences)
    query_embedding = model.encode(query_sentence)

    cosine_scores = util.cos_sim(query_embedding, embeddings)
    top_indices = cosine_scores.argsort()[0][-5:].flip(dims=[0]) # Get indices of top 5 most similar

    top_similar_sentences = [sentences[i] for i in top_indices]
    return top_similar_sentences


if __name__ == "__main__":
    topic = input("Enter the Wikipedia topic (e.g., 'Pakistan'): ")
    query = input("Enter your query sentence: ")

    results = semantic_similarity_query(topic, query)

    if results:
        print("\nTop 5 most similar sentences:")
        for i, sentence in enumerate(results):
            print(f"{i+1}. {sentence}")
    else:
        print("No similar sentences found.")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Enter the Wikipedia topic (e.g., 'Pakistan'): Amitabh Bachan 
Enter your query sentence: Superhit films

Top 5 most similar sentences:
1. Despite its heavy theme, Kabhi Kabhie went on to become a superhit.
2. In 1999, BBC India declared Sholay the "Film of the Millennium" and, like Deewaar, it has been cited by Indiatimes Movies as among the Top 25 Must See Bollywood Films.
3. He was exceptional, a genius actor who was in films that weren't good."
4. The film opened to excellent response all over the country, eventually taking top spot at the box office that year and emerging an All Time Blockbuster as well as Bachchan's biggest hit at that point of time.
5. The film emerged a huge blockbuster at the box office and earned him another Filmfare nomination for Best Actor.
